## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import pandas as pd
from utils import BankofCanadaRates, Transaction, Asset, Holdings
from IPython.display import display, HTML

displayPandas = lambda df: display(HTML(df.to_html()))

In [34]:
transactions_filepath = "transactions.csv"

trxs_log = pd.read_csv(transactions_filepath)
trxs_log.columns = map(lambda x: x.lower().replace(" ", "_"), trxs_log.columns)
trxs_log.index.name = "id"
trxs_log = trxs_log.sort_values(["date", "id"]).fillna({"description": "", "fees": 0, "note": ""})
displayPandas(trxs_log.tail(5))

assert trxs_log.isnull().sum().sum() == 0
trxs = trxs_log.apply(lambda r: Transaction(**r), axis=1).tolist()

,date,description,base_currency,quote_currency,quantity,price,fees,note
id,,,,,,,,
14,2023-11-10,,EXPE,USD,-89.0,118.685,0.0,
15,2023-11-15,RSU Vest,EXPE,USD,10.0,129.820,0.0,
16,2023-11-15,RSU Vest,EXPE,USD,9.0,129.820,0.0,
23,2024-01-17,RDI,DLR,USD,1041.0,10.100,0.0,
24,2024-01-17,RDI,DLR,CAD,-1041.0,13.630,0.0,


## Process

In [7]:
import json
from dataclasses import asdict

holdings = Holdings()
capgains = []
boc = BankofCanadaRates(start_date="2018-01-01")
reporting_currency = "CAD"

# artificial initial cash balance
holdings.add(Asset(trxs_log["date"].min(), reporting_currency, quantity=50000, acb=1))


def process_transaction(trx: Transaction):
    # reflect fees in the price
    trx = trx.with_effective_price()

    if trx.quote_to_reporting_rate is None:
        try:
            if trx.quote_currency == reporting_currency:
                trx.quote_to_reporting_rate = 1
            else:
                # retrieve the exchange rate from Bank of Canada
                trx.quote_to_reporting_rate = boc.get_rate(
                    trx.quote_currency, reporting_currency, trx.date
                )
        except:
            raise Exception(
                f"Error getting rate for {trx.date} {trx.quote_currency} to {reporting_currency}"
            )

    # Vesting transactions are fundeed by the company, so we need to add 
    # a preceeding funding transaction
    if "Vest" in trx.description:
        process_transaction(
            Transaction(
                date=trx.date,
                description=trx.description.replace("Vest", "Funding"),
                base_currency=trx.quote_currency,
                quote_currency=reporting_currency,
                quantity=trx.cost,
                price=trx.quote_to_reporting_rate,
                fees=0,
                quote_to_reporting_rate=1,
            )
        )
    
    # Flip the transaction if it is a sell order
    if trx.quantity < 0:
        trx = trx.flip()

    # Get the current holdings
    base_holding = holdings.get(trx.base_currency, trx.date)
    quote_holding = holdings.get(trx.quote_currency, trx.date)
    
    # Update the ACB and quantity for the base currency
    if trx.base_currency != reporting_currency:
        base_holding.acb = (
            base_holding.quantity * base_holding.acb + trx.cost * quote_holding.acb
        ) / (base_holding.quantity + trx.quantity)
    base_holding.quantity += trx.quantity
    base_holding.date = trx.date

    # Calculate the capital gain for liquidating transactions
    if trx.base_currency == reporting_currency:
        cost_baes = quote_holding.acb * trx.cost
        gross_proceeds = trx.quantity
        capital_gain = gross_proceeds - cost_baes
        capgains.append(
            {
                "Date": trx.date,
                "Gross Proceeds": gross_proceeds,
                "Cost Base": cost_baes,
                "Capital Gain": capital_gain,
            }
        )

    # Update the quantity for the quote currency
    if quote_holding.quantity < trx.cost:
        raise Exception(
            f"Insufficient funds to complete transaction on {trx.date}. Details:\n"
            f"Transaction: {json.dumps(asdict(trx), indent=4)}\n"
            f"Current holdings:\n"
            f"{holdings.current}"
        )
    quote_holding.quantity -= trx.cost
    quote_holding.date = trx.date

    holdings.add(base_holding, overwrite=True)
    holdings.add(quote_holding, overwrite=True)


for trx in trxs:
    process_transaction(trx)

## Status

In [28]:
print("Capital Gains")
displayPandas(pd.DataFrame(capgains))

print("Capital Gains: 2023")
displayPandas(
    pd.DataFrame(capgains)
    .query("'2023-01-01' <= Date <= '2023-12-31'")
    .drop(columns="Date")
    .sum()
    .to_frame()
    .T
)

print("Holdings")
displayPandas(holdings.current)

Capital Gains


,Date,Gross Proceeds,Cost Base,Capital Gain
0,2022-11-25,6556.297216,6729.220750,-172.923534
1,2023-02-28,6458.850000,6334.371282,124.478718
2,2023-02-28,13.880000,13.622304,0.257696
3,2023-10-19,18554.536000,17705.509986,849.026014
4,2024-01-17,14188.830000,12533.464954,1655.365046


Capital Gains: 2023


,Gross Proceeds,Cost Base,Capital Gain
0,25027.266,24053.503571,973.762429


Holdings


,date,asset,quantity,acb
40,2023-11-15,EXPE,99.000000,139.601397
43,2024-01-17,DLR,0.000000,12.039832
42,2024-01-17,CAD,38560.755239,1.000000
44,2024-01-17,USD,62.841000,1.192063


## Holdings Time Series

In [7]:
import plotly.express as px
from plotly.subplots import make_subplots

fig = px.line(holdings.df, x="date", y="quantity", color="asset", markers=True)
fig2 = px.line(holdings.df, x="date", y="acb", color="asset")
fig2.update_traces(yaxis="y2", line=dict(dash="dashdot"))

subfig = make_subplots(specs=[[{"secondary_y": True}]])
subfig.add_traces(fig.data + fig2.data)
subfig.layout.xaxis.title="Date"
subfig.layout.yaxis2.title="ACB (CAD)" + " --"
subfig.layout.yaxis.title="Quantity"
subfig.show()